In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

naw


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

### 2) load from raw file

In [3]:
#raw_file = "data/animalia_raw.xml"
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

All done loading raw file.


In [4]:
#pgd.parse_raw()
#pgd.write_prep_data("data/primate_df.csv", "data/primate_links.txt")

### 3) load from prepared files

In [3]:
pgd.load_prep_data("data/primate_df.csv", "data/primate_links.txt")

### return data for plotting

In [7]:
df, links_list = pgd.return_data()

## Plot Data

In [8]:
pgp = ph.phyloGraph(df, links_list)

must pass BOTH username & api_key in order to publish.


In [9]:
pgp.search_name('rrh')

             name     id
1     Platyrrhini  16292
18     Catarrhini  16293
83  Strepsirrhini  16295


In [10]:
pgp.create_plot_data(16293)

/Users/Seth/Documents/tolGraph/phyloGraph.py:234: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting 16293


In [12]:
pgp.render_plot()